In [1]:
pip install requests beautifulsoup4 pandas openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


# Path to the Excel file
file_path = r"C:\Users\Sandeep\Desktop\Input.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Print the column names to see what is available
print(df.columns)

df = pd.read_excel(r"C:\Users\Sandeep\Desktop\Input.xlsx") 


def extract_article(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        title = soup.find('h1').get_text() if soup.find('h1') else "No Title"
       
        paragraphs = soup.find_all('p')
        article_text = "\n".join([p.get_text() for p in paragraphs])
        
        return title, article_text
    else:
        return "Error", "Failed to fetch article"


for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    
    title, article_text = extract_article(url)
    
    with open(f"{url_id}.txt", "w", encoding="utf-8") as file:
        file.write(f"Title: {title}\n\n")
        file.write(article_text)
    print(f"Article for {url_id} saved successfully.")


Index(['URL_ID', 'URL'], dtype='object')
Article for bctech2011 saved successfully.


In [5]:
import pandas as pd
import os
import re
from textblob import TextBlob
import nltk

nltk.download('punkt')

def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    return polarity, subjectivity

def avg_sentence_length(text):
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)
    return len(words) / len(sentences)

def percentage_complex_words(text):
    words = nltk.word_tokenize(text)
    complex_words = [word for word in words if len(re.findall(r'[aeiou]', word)) > 2]
    return len(complex_words) / len(words) * 100

def fog_index(text):
    avg_sentence_len = avg_sentence_length(text)
    percent_complex_words = percentage_complex_words(text)
    return 0.4 * (avg_sentence_len + percent_complex_words)

def syllables_per_word(text):
    vowels = "aeiou"
    words = nltk.word_tokenize(text)
    syllable_count = sum([sum(1 for char in word if char in vowels) for word in words])
    return syllable_count / len(words) if words else 0

def complex_word_count(text):
    words = nltk.word_tokenize(text)
    complex_words = [word for word in words if len(re.findall(r'[aeiou]', word)) > 2]
    return len(complex_words)

def personal_pronouns_count(text):
    pronouns = ["I", "we", "you", "he", "she", "it", "they"]
    return sum([text.lower().count(pronoun.lower()) for pronoun in pronouns])

def word_count(text):
    words = nltk.word_tokenize(text)
    return len(words)

def avg_word_length(text):
    words = nltk.word_tokenize(text)
    return sum(len(word) for word in words) / len(words) if words else 0

# Path to the Excel file
file_path = r"C:\Users\Sandeep\Desktop\Input.xlsx"

# Read Excel data (assuming the column containing text is named 'Text')
df = pd.read_excel(file_path)

results = []
for index, row in df.iterrows():
    text = row['Text']  # Assuming 'Text' column contains the text data
    sentiment = analyze_sentiment(text)
    analysis = {
        'File Name': f'Row {index}',  # Use row index as file identifier
        'Positive Score': sentiment[0],
        'Negative Score': sentiment[1],
        'Polarity Score': (sentiment[0] - sentiment[1]) / (abs(sentiment[0]) + abs(sentiment[1]) + 0.0001),
        'Subjectivity Score': (sentiment[0] + sentiment[1]) / (len(text.split()) + 0.0001),
        'Average Sentence Length': avg_sentence_length(text),
        'Percentage of Complex Words': percentage_complex_words(text),
        'Fog Index': fog_index(text),
        'Syllable per Word': syllables_per_word(text),
        'Complex Word Count': complex_word_count(text),
        'Word Count': word_count(text),
        'Personal Pronouns': personal_pronouns_count(text),
        'Average Word Length': avg_word_length(text),
    }
    results.append(analysis)

# Convert to DataFrame and save to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("text_analysis_results.csv", index=False)
print("Analysis saved to text_analysis_results.csv")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sandeep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyError: 'Text'